# Labor 1

A párhuzamos programozás olyan programozási paradigma, amely lehetővé teszi, hogy egyidejűleg több feladatot hajtson végre egy számítógép vagy több számítógép egyszerre. A párhuzamos programozás lehetővé teszi a számítási teljesítmény növelését, a nagyobb vagy bonyolultabb számítási feladatok megoldását.

A [GPGPU](https://www.gigabyte.com/Glossary/gpgpu) rövidítés a "General Purpose Graphics Processing Unit"-ra utal, ami egy olyan számítási technológia, amely során a grafikus kártyák (GPU-k) számítási kapacitását használjuk általános célú számítások végrehajtására. A GPU-k eredetileg kifejezetten grafikus feladatok hatékony megoldására lettek tervezve, mint például a képfeldolgozás vagy 3D grafika. Azonban, az elmúlt években a GPU-k egyre inkább beépültek az általános célú számítások világába is.

A GPGPU technológia alkalmazása lehetővé teszi, hogy a GPU-k nagy számítási kapacitását felhasználják olyan feladatok elvégzésére, mint például a gépi tanulás, a kriptográfiai műveletek, a tudományos szimulációk vagy a nagy adathalmazok feldolgozása. A GPGPU technológia használata jelentősen felgyorsíthatja ezeket a számítási feladatokat, és csökkentheti azok elvégzésének idejét.

## CUDA Runtime API


A [CUDA Runtime API](https://docs.nvidia.com/cuda/cuda-runtime-api/index.html) egy függvénykönyvtárból és a C++ szintaxis egyszerű kiegészítéséből áll.

A CUDA programokat `*.cu` (nem `*.c` vagy `*.cpp`) kiterjesztésű állományokban írjuk. Mint meglátjuk, alapvetően C++ kódot írunk, melyet néha kiegészíthetünk gyorsítón futtatandó programrészletekkel, úgynevezett "CUDA kernelekkel".

Az alábbi kódrészlet egy tökéletesen működő CUDA program, csak éppenséggel még nem tartalmaz gyorsító specifikus kódot.

```cpp
#include <iostream>

int main(int argc, char** argv)
{
    std::cout << "Hello World!" << std::endl;
    return 0;
}
```


Ha a programunkban szeretnénk kihasználni a GPU számítási kapacitását lehetőségeit is, akkor GPU kódot is írnunk kell. Ez a számításokhoz szükséges adatokat előkészíti, átmásolja, meghívja a GPU kernelt, majd az adatokat visszamásolja. A CUDA forráskódok általában keverten tartalmazzák a CPU, és GPU kódokat.

A GPU-n futó számítást úgynevezett a fentebb említett kernel függvények megadásával tudjuk megvalósítani. Ezek a `__global__` előtaggal rendelkező függvények.

### Kompilálás és futtatás

A CUDA (`*.cu` kiterjesztésű) programjaink kompilálásához, használjuk a következő parancsot:  

```
!nvcc mysurcefile.cu -o programname
```

majd sikeres kompilálás esetében, futtatáshoz:
```
!./programname
```

## Google Colaboratory

A Google Colaboratory (röviden Colab) egy ingyenes online platform, amely lehetővé teszi a felhasználók számára a (főleg) Python programozási nyelv interaktív környezetében történő munkavégzését. Az egyik fő előnye, hogy ingyenes GPU és TPU (Tensor Processing Unit) számítási erőforrásokat biztosít a felhasználók számára, azzal a cállal, hogy lehetővé tegye a nagyobb adatméretű és bonyolultabb gépi tanulási és adatelemzési feladatok végrehajtását.

Az GPU erőforrások igénybevétele érdekében, a felhasználóknak a futásidejű környezet típusát kell áttálítsa. Ehhez, a `Runtime/Change runtime` menűpontban válasszuk ki, hogy `GPU`. 

A `GPU` elérhetőségét tesztelni tudjuk a következő kóddal: 

In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


A Nvidia kompájler verziójának lekérése:

In [9]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Tue_Mar__8_18:18:20_PST_2022
Cuda compilation tools, release 11.6, V11.6.124
Build cuda_11.6.r11.6/compiler.31057947_0


A kód cellákkban csak Python kód futtatható direkt modon. Ezért, [%%writefile magic](https://ipython.readthedocs.io/en/stable/interactive/magics.html) parancsal fogjuk kiírni a lemezre a programjainkat, majd ezeket más kódcellákban kompiláljuk és futtatjuk.

Például, az alábbi példaprogram GPU-n végzi el két szám összeadását.

In [4]:
%%writefile vecadd.cu
#include <cuda.h>
#include <stdio.h>

__global__ void add(int a, int b, int* c)
{
    *c = a + b;
    return;
}

int main(int argc, char** argv)
{
    int c;
    int* dev_c;
	
	//memória foglalás a GPU-n
    cudaMalloc((void**) &dev_c, sizeof(int));

    add<<<1,1>>>(1, 2, dev_c);

    cudaMemcpy(&c, dev_c, sizeof(int), cudaMemcpyDeviceToHost);

    printf("a + b = %d\n", c);
    
    //lefolglat memória felszabadítása
    cudaFree(dev_c);
    return 0;
}

Overwriting vecadd.cu


És íme meg is jelenik a `vecadd.cu` állomány:

In [6]:
!ls

sample_data  vecadd.cu


Kompilálás:

In [7]:
!nvcc vecadd.cu

Ellenörizzük ha megjelent a futtatható bináris program (`a.out`):

In [5]:
!ls

a.out  sample_data  vecadd.cu


Futtatás:

In [8]:
!./a.out

a + b = 3


# Feladatok

1. Írjunk egy CUDA programot, mely a [`cudaError_t cudaGetDeviceCount (int * count)`](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__DEVICE.html#group__CUDART__DEVICE_1g18808e54893cfcaafefeab31a73cc55f), [`cudaError_t cudaGetDeviceProperties (struct cudaDeviceProp * prop, int device )`](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__DEVICE.html#group__CUDART__DEVICE_1g1bf9d625a931d657e08db2b4391170f0) függvények segítségével, kiírja a Rocks szerveren található CUDA kompatibilis GPU főbb paramétereit.
A jellemzőket tartalmazó `cudaDeviceProp` struktúra definíciója a következő:

  ```cpp
  struct cudaDeviceProp {
                char name[256];
                cudaUUID_t uuid;
                size_t totalGlobalMem;
                size_t sharedMemPerBlock;
                int regsPerBlock;
                int warpSize;
                size_t memPitch;
                int maxThreadsPerBlock;
                int maxThreadsDim[3];
                int maxGridSize[3];
                int clockRate;
                size_t totalConstMem;
                int major;
                int minor;
                size_t textureAlignment;
                size_t texturePitchAlignment;
                int deviceOverlap;
                int multiProcessorCount;
                int kernelExecTimeoutEnabled;
                int integrated;
                int canMapHostMemory;
                int computeMode;
                int maxTexture1D;
                int maxTexture1DMipmap;
                int maxTexture1DLinear;
                int maxTexture2D[2];
                int maxTexture2DMipmap[2];
                int maxTexture2DLinear[3];
                int maxTexture2DGather[2];
                int maxTexture3D[3];
                int maxTexture3DAlt[3];
                int maxTextureCubemap;
                int maxTexture1DLayered[2];
                int maxTexture2DLayered[3];
                int maxTextureCubemapLayered[2];
                int maxSurface1D;
                int maxSurface2D[2];
                int maxSurface3D[3];
                int maxSurface1DLayered[2];
                int maxSurface2DLayered[3];
                int maxSurfaceCubemap;
                int maxSurfaceCubemapLayered[2];
                size_t surfaceAlignment;
                int concurrentKernels;
                int ECCEnabled;
                int pciBusID;
                int pciDeviceID;
                int pciDomainID;
                int tccDriver;
                int asyncEngineCount;
                int unifiedAddressing;
                int memoryClockRate;
                int memoryBusWidth;
                int l2CacheSize;
                int persistingL2CacheMaxSize;
                int maxThreadsPerMultiProcessor;
                int streamPrioritiesSupported;
                int globalL1CacheSupported;
                int localL1CacheSupported;
                size_t sharedMemPerMultiprocessor;
                int regsPerMultiprocessor;
                int managedMemory;
                int isMultiGpuBoard;
                int multiGpuBoardGroupID;
                int singleToDoublePrecisionPerfRatio;
                int pageableMemoryAccess;
                int concurrentManagedAccess;
                int computePreemptionSupported;
                int canUseHostPointerForRegisteredMem;
                int cooperativeLaunch;
                int cooperativeMultiDeviceLaunch;
                int pageableMemoryAccessUsesHostPageTables;
                int directManagedMemAccessFromHost;
                int accessPolicyMaxWindowSize;
            }
   ```

2. Írjunk egy "Hello, World" CUDA kernelt, melyben minden szál kiírja az [egyedi azonosítóját](https://blog.usejournal.com/cuda-thread-indexing-fb9910cba084). Hívjuk meg a kernelt különböző rács és blokk konfigurációkkal. Használjuk a [cudaError_t cudaDeviceSynchronize ( void )](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__DEVICE.html#group__CUDART__DEVICE_1g10e20b05a95f638a4071a655503df25d) függvényt a kernel hívás bevárására.


# 1. feladat megoldása

In [47]:
%%writefile deviceProps.cu
#include <cuda.h>
#include <stdio.h>

int main(int argc, char** argv)
{
    int deviceId;
    cudaGetDevice(&deviceId);

    cudaDeviceProp props;
    cudaGetDeviceProperties(&props, deviceId);

    char *deviceName = props.name;
    int computeCapabilityMajor = props.major;
    int computeCapabilityMinor = props.minor;
    int multiProcessorCount = props.multiProcessorCount;
    int warpSize = props.warpSize;

    printf("Device name: %s\nDevice ID: %d\nNumber of SMs: %d\nCompute Capability Major: %d\nCompute Capability Minor: %d\nWarp Size: %d\n", deviceName, deviceId, multiProcessorCount, computeCapabilityMajor, computeCapabilityMinor, warpSize);

    return 0;
}

Overwriting deviceProps.cu


In [48]:
!nvcc deviceProps.cu -o deviceProps -run

Device name: Tesla T4
Device ID: 0
Number of SMs: 40
Compute Capability Major: 7
Compute Capability Minor: 5
Warp Size: 32


# 2. feladat megoldás

In [27]:
%%writefile helloWorld.cu
#include <cuda.h>
#include <stdio.h>

__global__ void helloWorld()
{
    printf("Block id: %d -- Thread id: %d\n", blockIdx.x, threadIdx.x);
}

int main(int argc, char** argv)
{
    helloWorld<<<1,5>>>();
    cudaDeviceSynchronize();

    helloWorld<<<8,16>>>();
    cudaDeviceSynchronize();
    return 0;
}

Overwriting helloWorld.cu


In [28]:
!nvcc helloWorld.cu -o helloWorld -run

Block id: 0 -- Thread id: 0
Block id: 0 -- Thread id: 1
Block id: 0 -- Thread id: 2
Block id: 0 -- Thread id: 3
Block id: 0 -- Thread id: 4
Block id: 0 -- Thread id: 0
Block id: 0 -- Thread id: 1
Block id: 0 -- Thread id: 2
Block id: 0 -- Thread id: 3
Block id: 0 -- Thread id: 4
Block id: 0 -- Thread id: 5
Block id: 0 -- Thread id: 6
Block id: 0 -- Thread id: 7
Block id: 0 -- Thread id: 8
Block id: 0 -- Thread id: 9
Block id: 0 -- Thread id: 10
Block id: 0 -- Thread id: 11
Block id: 0 -- Thread id: 12
Block id: 0 -- Thread id: 13
Block id: 0 -- Thread id: 14
Block id: 0 -- Thread id: 15
Block id: 5 -- Thread id: 0
Block id: 5 -- Thread id: 1
Block id: 5 -- Thread id: 2
Block id: 5 -- Thread id: 3
Block id: 5 -- Thread id: 4
Block id: 5 -- Thread id: 5
Block id: 5 -- Thread id: 6
Block id: 5 -- Thread id: 7
Block id: 5 -- Thread id: 8
Block id: 5 -- Thread id: 9
Block id: 5 -- Thread id: 10
Block id: 5 -- Thread id: 11
Block id: 5 -- Thread id: 12
Block id: 5 -- Thread id: 13
Block id: 